In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import os
import psycopg2

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("DeltaTest") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


In [ ]:
aws_access_key = os.getenv("AWS_ACCESS_KEY")
aws_secret = os.getenv("AWS_SECRET_ACCESS_KEY_ID")

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", aws_access_key)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", aws_secret)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3n.endpoint", "s3.amazonaws.com")

In [ ]:
df = spark.read.parquet("s3n://covid-delta-lake/test/cases/*.parquet")

In [ ]:
df.show(10)

In [ ]:
df.printSchema()

In [ ]:
df_testing = spark.read.parquet("s3n://covid-delta-lake/delta/tests/*.parquet")

In [ ]:
df_testing.printSchema()

In [ ]:
df_testing.select("date","state","positive", "negative", "death","totalTestResults").show()

In [ ]:
df_populations = spark.read.parquet("s3n://covid-delta-lake/delta/populations/*.parquet")

In [ ]:
df_populations.show(10)

In [ ]:
df_populations.printSchema()

In [ ]:
df_states= spark.read.option("header", True).csv("s3n://covid-delta-lake/static/*.csv")

In [ ]:
df_states.printSchema()

In [ ]:
df_states = df_states.withColumnRenamed("State", "state_full")
df_states.show(10)

In [ ]:
df = df.join(df_states.select("state_full", "Code"), df_states["state_full"] == df["state"], "inner").select("date", "county", "state_full", "fips", "cases", "deaths", "code")

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('redshift.cfg')

In [ ]:
from sql_queries import population_table_create, copy_table_population
import psycopg2

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [ ]:
drop = ("DROP TABLE IF EXISTS populations")

In [ ]:
cur.execute(drop)
conn.commit()

In [ ]:
pop_create_query= ("""CREATE TABLE IF NOT EXISTS populations( 
                                    Id varchar, 
                                    Id2 bigint, 
                                    County varchar, 
                                    state varchar, 
                                    pop_estimate_2018 bigint);""")

In [ ]:
cur.execute(pop_create_query)
conn.commit()

In [ ]:
pop_copy_query = ("""
    COPY populations 
    FROM 's3://covid-delta-lake/delta/populations/parquets'
    IAM_ROLE '{}' 
    FORMAT AS PARQUET;
    """).format(config.get("IAM_ROLE", "ARN"))

In [ ]:
cur.execute(pop_copy_query)
conn.commit()

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [ ]:
from sql_queries import covid_table_create, copy_table_cases

In [ ]:
drop_cases = ("DROP TABLE IF EXISTS covid_cases")

In [ ]:
cur.execute(drop_cases)
conn.commit()

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS covid_cases(
                            date VARCHAR,
                            county VARCHAR,
                            state_full VARCHAR,
                            fips DOUBLE PRECISION,
                            cases BIGINT,
                            deaths BIGINT,
                            code VARCHAR                           
                                );""")
conn.commit()

In [ ]:
copy_table_cases2 = ("""
    COPY covid_cases
    FROM 's3://covid-delta-lake/delta/cases/parquets'
    IAM_ROLE '{}'
    FORMAT AS PARQUET;
    """).format(config.get("IAM_ROLE", "ARN"))

In [ ]:
cur.execute(copy_table_cases2)
conn.commit()

In [ ]:
tests_table_create= ("""CREATE TABLE IF NOT EXISTS covid_tests(
                            date BIGINT,
                            state_full VARCHAR,
                            positive DOUBLE PRECISION,
                            negative DOUBLE PRECISION,
                            death DOUBLE PRECISION,
                            total DOUBLE PRECISION,
                            hash VARCHAR,
                            dateChecked VARCHAR,
                            totalTestResults DOUBLE PRECISION,
                            fips BIGINT,
                            deathIncrease BIGINT,
                            hospitalizedIncrease BIGINT,
                            negativeIncrease BIGINT,
                            positiveIncrease BIGINT,
                            totalTestResultsIncrease BIGINT,
                            hospitalized DOUBLE PRECISION,
                            pending DOUBLE PRECISION                           
                                );""")

In [ ]:
cur.execute(tests_table_create)
conn.commit()

In [ ]:
copy_table_tests = ("""
    COPY covid_tests 
    FROM 's3://covid-delta-lake/delta/tests/parquets'
    IAM_ROLE '{}'
    FORMAT AS PARQUET;
    """).format(config.get("IAM_ROLE", "ARN"))

In [ ]:
cur.execute(copy_table_tests)
conn.commit()

In [ ]:
😂😂

In [ ]:
time_table_create = ("""CREATE TABLE IF NOT EXISTS time(
                        date VARCHAR,
                        date_ts DATE,
                        day INT, 
                        week INT, 
                        month INT,
                        year INT,
                        weekday INT);
                    """)

In [ ]:
time_table_insert = ("""
INSERT INTO time (
                date,
                date_ts, 
                day, 
                week, 
                month,
                year,
                weekday)
SELECT  date as date,
        TO_DATE(date, 'YYYY-MM-DD') as date_ts,
        EXTRACT(day FROM date_ts) as day,
        EXTRACT(week FROM date_ts) as week,
        EXTRACT(month FROM date_ts) as month,
        EXTRACT(year FROM date_ts) as year,
        EXTRACT(weekday FROM date_ts) as weekday
FROM(
  SELECT DISTINCT date
  FROM covid_cases
  WHERE date IS NOT null)
""")

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [ ]:
cur.execute("DROP TABLE IF EXISTS time")
conn.commit()

In [ ]:
cur.execute(time_table_create)
conn.commit()

In [ ]:
cur.execute(time_table_insert)
conn.commit()

In [ ]:
config = configparser.ConfigParser()
config.read('redshift.cfg')

In [ ]:
cur.execute("select count(*)from covid_tests")
result = cur.fetchall()

In [ ]:
result[0][0]

In [ ]:
for table in ["covid_tests", "covid_cases", "populations", "time"]:
    cur.execute(f"select count(*) from {table}")
    result = cur.fetchall()
    if result[0][0] > 1:
        print("Data quality inspection passed")
    else:
        raise ValueError(f"QA failed for {table}: Table contained 0 rows")

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(host, db_name, user, pw, port))
cur = conn.cursor()
print("connected")

In [ ]:
cur.execute(f"select MAX(year), MIN(year) from time")
result = cur.fetchall()

In [ ]:
result

In [ ]:
type(result[0][0])

In [ ]:
result[0][1]

In [ ]:
cur.execute(f"select MAX(year), MIN(year) from time")

result = cur.fetchall()
max_year = result[0][0]
min_year = result[0][1]

if max_year <= 2022:
    if min_year >= 2019:
        print(f"Data quality inspection passed for time table")
    else:
        raise ValueError(f"QA failed for time check. The minimum year value returned was {min_year}, which is outside the time range")
else:
    raise ValueError(f"QA failed for time check. The maximum year value returned was {max_year}, which is outside the time range")

In [ ]:
max_year

In [ ]:
min_year

In [ ]:
conn.close()

In [ ]:
tables = ["covid_tests", "covid_cases", "populations", "time"]
for table in tables:

    cur.execute(f"select count(*) from {table}")
    result = cur.fetchall()
    if result[0][0] > 1:
        print(f"Data quality inspection passed for {table}")
    else:
        raise ValueError(f"QA failed for {table}: Table contained 0 rows. Please clear all Redshit tables and re-run script")